In [7]:
using Clapeyron
model = PCSAFT(["water","methanol","hexane"];userlocations=(;
Mw = [18.015,32.042,86.18], 
segment = [2.1945,1.5255,3.0576],
sigma = [2.229,3.23,3.7983],
epsilon = [141.66,188.9,236.77],
n_H=[2,1,0],
n_e=[2,1,0],
epsilon_assoc = Dict(
    (("methanol","e"),("methanol","H")) =>2899.5,
    (("methanol","H"),("water","e")) =>2351.835,
	(("methanol","e"),("water","H")) =>2351.835,
    (("water","H"),("water","e")) =>1804.17
    ),
bondvol = Dict(
    (("methanol","e"),("methanol","H")) =>0.035176,
    (("methanol","H"),("water","e")) =>0.080454702,
	(("methanol","e"),("water","H")) =>0.080454702,
    (("water","H"),("water","e")) =>0.2039
)
))


PCSAFT{BasicIdeal, Float64} with 3 components:
 "water"
 "methanol"
 "hexane"
Contains parameters: Mw, segment, sigma, epsilon, epsilon_assoc, bondvol

In [8]:
function SpeciesFugacityCheck(model::EoSModel, T, p, x)
    numphases, numspecies = size(x)
    Fugacities = zeros(numphases, numspecies)
    for i = 1:numphases
        Fugacities[i,:] = fugacity_coefficient(model, p, T, x[i,:]) .* x[i,:] .* p
    end  
    RelErrorSpecies = zeros(numspecies)
    for j = 1:numspecies
        RelErrorSpecies[j] = (sum(abs(Fugacities[1,j] - Fugacities[i,j])/Fugacities[1,j] for i in 2:numphases))/(numphases-1)
    end  
    normRelError = sum(abs.(RelErrorSpecies)) / numspecies  
    return (normRelError, RelErrorSpecies, Fugacities)
end


SpeciesFugacityCheck (generic function with 1 method)

In [9]:
N = 1000
T = 298.15
p = 1.013e5
z0 = [0.4999,0.0002,0.4999]

K0 = [0.001,10,1000]
x_LLE = zeros(N,6)
idxend = N
(x,n,G) = tp_flash(model,p,T,z0,MichelsenTPFlash(equilibrium=:lle,K0=K0))
(normRelError_1PC, RelErrorSpecies_1PC, Fugacities_1PC) = SpeciesFugacityCheck(model, 298.15, 1.013e5, x)


(3.0992201129745475e-9, [2.840066769274575e-9, 6.455074550541373e-9, 2.519019107695468e-12], [3178.183657528801 12.59338521488834 20029.67262988092; 3178.183666555055 12.59338529617958 20029.672629830464])

In [10]:
x_LLE[1,1:3] = x[1,:]
x_LLE[1,4:6] = x[2,:]
K0 = x[1,:]./x[2,:]
z0[1] = x[1,1]/2+x[2,1]/2
z0[2] += 2/N
z0[3] = x[1,3]/2+x[2,3]/2
for i in 2:N
    global z0, K0, x_LLE, idxend
    (x,n,G) = tp_flash(model,p,T,z0,MichelsenTPFlash(equilibrium=:lle,K0=K0))
    x_LLE[i,1:3] = x[1,:]
    x_LLE[i,4:6] = x[2,:]
    K0 = x[1,:]./x[2,:]

    z0 = (x_LLE[i,1:3]+x_LLE[i,4:6])-(x_LLE[i-1,1:3]+x_LLE[i-1,4:6])/2
    if abs(x[1,1]-x[2,1]) < 1e-3
        idxend=i-1
        break
    end
end

x_LLE = vcat(x_LLE[1:idxend,1:3],reverse(x_LLE[1:idxend,4:6]));



428×3 Matrix{Float64}:
 0.989869  0.000391768  0.009739
 0.985867  0.0043289    0.00980399
 0.981864  0.00826428   0.00987173
 0.97786   0.0121978    0.00994223
 0.973855  0.0161296    0.0100155
 0.969849  0.0200594    0.0100916
 0.965842  0.0239872    0.0101704
 0.961835  0.027913     0.0102521
 0.957827  0.0318368    0.0103366
 0.953818  0.0357584    0.0104239
 ⋮                      
 0.998438  0.000446821  0.00111469
 0.998508  0.000384751  0.00110747
 0.998575  0.00032473   0.00110043
 0.99864   0.000266705  0.00109356
 0.998703  0.000210626  0.00108685
 0.998763  0.000156441  0.0010803
 0.998822  0.000104101  0.00107391
 0.998879  5.35567e-5   0.00106768
 0.998934  4.76055e-6   0.0010616